# Generate QnA synthetic dataset from a Complex PDF using Unstructured

### Overview
We process the PDF by dividing it into three parts.

- **Text-heavy** - Text-heavy PDF can be processed with open source without the need to use toolkits like Azure AI Document Intelligence or Unstructured.
- **Image-heavy** - Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.
- **Mixed** - After reading the document with Azure AI Document Intelligence, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)


## 1. Read & Preprocess PDF file
---

### Split the PDFs into individual pages
Only use a poration of the PDF documents for testing

In [1]:

import os, shutil, random
import openai
from dotenv import load_dotenv
from unstructured.cleaners.core import clean_bullets, clean_extra_whitespace, remove_punctuation
from langchain_community.document_loaders import UnstructuredFileLoader, UnstructuredMarkdownLoader, UnstructuredAPIFileLoader
from langchain_community.document_loaders.csv_loader import CSVLoader, UnstructuredCSVLoader

load_dotenv()

raw_data_dir = "raw_data"
splitted_raw_data_dir = f"splitted_{raw_data_dir}"

#file_path = f"{raw_data_dir}/prod-unst-pdf/[Sales Talk] 3. QnA3_Handling Objection_(S24)_240227.pdf"
file_path = f"{raw_data_dir}/prod-unst-pdf/SM-S92X_UG_UU_Kor_Rev.1.1_240129.pdf"


In [2]:
import fitz

# Open the first PDF document
doc1 = fitz.open(file_path)
split_pages = [(1, 15)]

for idx, s in enumerate(split_pages):
    # Create a new empty PDF document
    doc2 = fitz.open()

    # Insert the first 2 pages of doc1 into doc2
    doc2.insert_pdf(doc1, from_page=s[0], to_page=s[1])

    # Save the modified document
    doc2.save(f"{raw_data_dir}/prod-unst-pdf/s24-user-manual-part{idx}.pdf")

In [3]:
from util.common_utils import delete_folder_and_make_folder
from util.preprocess import remove_short_sentences, remove_small_images, analyze_pdf_page_content, split_pdf

file_path = f"{raw_data_dir}/prod-unst-pdf/s24-user-manual-part0.pdf"
result = analyze_pdf_page_content(file_path)
delete_folder_and_make_folder(splitted_raw_data_dir)    

print("### PDF Content Analysis Result:")
for content_type, pages in result.items():
    print(f"{content_type} pages: {pages}")
    split_pdf(file_path, f"{splitted_raw_data_dir}/{content_type}.pdf", pages)

The folder 'splitted_raw_data' and its contents have been deleted.
### PDF Content Analysis Result:
Text pages: [0, 1, 8, 10, 12, 13]
Mixed pages: [2, 3, 4, 5, 6, 7, 11, 14]
Image pages: [9]


### Case 1: Mixed page (Images and text mixed appropriately)
After reading the document with UnstructuredFileLoader, we replace the image descriptions inside the figure tags with text summarized by a multimodal LLM. (Often the image descriptions are blank or have only a short caption.)


In [5]:
%%time

pdf_mixed_path = f"{splitted_raw_data_dir}/Mixed.pdf"

chunk_size = 1500
new_after_n_chars = 1200
combine_text_under_n_chars = 1000
chunk_overlap = 100
max_tokens = 1024
image_dir = "./images"

loader = UnstructuredFileLoader(
    file_path=pdf_mixed_path,

    chunking_strategy = "by_title",
    mode="elements",

    extract_image_block_types=["Image", "Table"],
    hi_res_model_name="yolox_quantized", #"detectron2_onnx", "yolox", "yolox_quantized"

    extract_images_in_pdf=True,
    #skip_infer_table_types='[]', # ['pdf', 'jpg', 'png', 'xls', 'xlsx', 'heic']
    skip_infer_table_types=True, ## enable to get table as html using tabletrasformer

    extract_image_block_output_dir=image_dir,
    extract_image_block_to_payload=False, ## False: to save image

    max_characters=chunk_size,
    new_after_n_chars=new_after_n_chars,
    combine_text_under_n_chars=combine_text_under_n_chars, # 이 문자 수 이하의 텍스트는 결합

    languages= ["kor+eng"],

    post_processors=[clean_bullets, clean_extra_whitespace, remove_punctuation]
)
docs = loader.load()

TypeError: argument of type 'bool' is not iterable

In [ ]:
images = remove_small_images(image_dir, image_dim_thres=16)
tables, texts = [], []

for doc in docs:
    category = doc.metadata["category"]
    if category == "Table": tables.append(doc)
    else: texts.append(doc)

print (f' # texts: {len(texts)} \n # tables: {len(tables)} \n # images: {len(images)}')

#### Summarize images

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=max_tokens,
    openai_api_version="2024-05-01-preview",
    azure_deployment="gpt-4o"                       
)

system_prompt = "You are an assistant tasked with describing table or image, specialized in Smartphone product."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": '''Given image, give a concise summary in Korean. Don't insert any XML tag such as <text> and </text> when answering.'''
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()
#summarize_chain = {"image_base64": lambda x:x} | prompt | llm_text | StrOutputParser()

In [ ]:
%%time
from util.preprocess import encode_image_base64
#images = glob(os.path.join(image_path, "*.jpg"))
base64_images = [encode_image_base64(img_path) for img_path in images]
image_summaries = summarize_chain.batch(base64_images, {"max_concurrency": 3})
image_summaries = remove_short_sentences(image_summaries)

In [ ]:
from util.preprocess import split_text_using_tiktoken

texts_tiktoken = split_text_using_tiktoken(texts, chunk_size, chunk_overlap)

mixed_chunks = image_summaries + texts_tiktoken
print("Length of splits (mixed case): " + str(len(mixed_chunks)))

### Case 2: Text-heavy
Text-heavy PDFs can be processed with open source without the need to use toolkits like Azure AI Document Intelligence or Unstructured.

In [ ]:
from langchain_community.document_loaders.pdf import PyMuPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

pdf_text_path = f"{splitted_raw_data_dir}/Text.pdf"
loader = PyMuPDFLoader(pdf_text_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, 
    chunk_overlap=200
)

text_chunks = text_splitter.split_documents(documents)

for idx, chunk in enumerate(text_chunks):
    print(f"Chunk {idx}\n{chunk}")
    print("="*80)
    if idx == 2:
        break

text_chunks = [d.page_content for d in text_chunks]
print("Length of splits (text-heay case): " + str(len(text_chunks)))

### Case 3: Image-heavy
Image-heavy PDF can be converted the entire page to images and let a multimodal LLM like GPT-4o summarize each page.

### Preprocess Image

In [ ]:
image_dir = "./pdf_image_tmp"
delete_folder_and_make_folder(image_dir) 

In [ ]:
import fitz
from glob import glob

pdf_image_path = f"{splitted_raw_data_dir}/Image.pdf"
doc = fitz.open(pdf_image_path)
#clip_x, clip_y = 10, 45
clip_x, clip_y = 10, 10

for i, page in enumerate(doc):
    x, y, w, h = page.rect
    clip = fitz.Rect(x+clip_x, y+clip_y, w-clip_x, h-clip_y)
    page.set_cropbox(clip)
    pix = page.get_pixmap()
    pix.save(f"{image_dir}/page_{i:03d}.jpg")

images = sorted(glob(os.path.join(image_dir, "*.jpg")))

In [ ]:
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain_openai import AzureChatOpenAI

max_tokens = 1024
llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=max_tokens,
    openai_api_version="2024-05-01-preview",
    azure_deployment="gpt-4o"                       
)

system_prompt = "You are an assistant tasked with describing table or image, specialized in Smartphone product."
system_message_template = SystemMessagePromptTemplate.from_template(system_prompt)
human_prompt = [
    {
        "type": "image_url",
        "image_url": {
            "url": "data:image/png;base64," + "{image_base64}",
        },
    },
    {
        "type": "text",
        "text": '''Given image, give a concise summary in Korean. Don't insert any XML tag such as <text> and </text> when answering.'''
    },
]
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

prompt = ChatPromptTemplate.from_messages(
    [
        system_message_template,
        human_message_template
    ]
)

summarize_chain = prompt | llm | StrOutputParser()
#summarize_chain = {"image_base64": lambda x:x} | prompt | llm_text | StrOutputParser()

In [ ]:
%%time
from util.preprocess import encode_image_base64
#images = glob(os.path.join(image_path, "*.jpg"))
base64_images = [encode_image_base64(img_path) for img_path in images]
image_summaries = summarize_chain.batch(base64_images, {"max_concurrency": 8})
image_summaries = remove_short_sentences(image_summaries)


In [ ]:
print("Length of image_summaries (image-heavy case): " + str(len(image_summaries)))

## 2. Construct QnA Pairs
----

### Option 1. 
Leverage the azure-ai-generative package. The QADataGenerator class in this package makes it easy to generate QnA synthetic questions. However, using this class as is has the disadvantage of not being able to use custom prompts, so we inherited from it and created the CustomQADataGenerator class.


In [ ]:
from util.qa import CustomQADataGenerator
model_config = {
    "deployment": os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    "model": "gpt-4o",
    "max_tokens": 2000,
}

qa_generator = CustomQADataGenerator(model_config=model_config, templates_dir="./prompt_template/ko")

In [ ]:
import asyncio
from collections import Counter
from typing import Dict
import os
from azure.ai.generative.synthetic.qa import QAType
concurrency = 6  # number of concurrent calls
sem = asyncio.Semaphore(concurrency)

#qa_type = QAType.CONVERSATION
qa_type = QAType.LONG_ANSWER

async def generate_async(text: str) -> Dict:
    async with sem:
        return await qa_generator.generate_async(
            text=text,
            qa_type=qa_type,
            num_questions=3,  # Number of questions to generate per text
        )

In [ ]:
input_batch = mixed_chunks + text_chunks + image_summaries
results = await asyncio.gather(*[generate_async(text) for text in input_batch], return_exceptions=True)

question_answer_list = []
token_usage = Counter()
for result in results:
    if isinstance(result, Exception):
        raise result  # exception raised inside generate_async()
    question_answer_list.append(result["question_answers"])
    token_usage += result["token_usage"]

print("Successfully generated QAs")

In [ ]:
question_answer_list[0]

### Option 2. 
You write the entire sequence of code to create a QnA dataset without using a separate toolkit. 

In [ ]:
aoai_api_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
aoai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from util.qa_pair import get_qna_prompt_template, QAPair

llm = AzureChatOpenAI(
    temperature=0, 
    max_tokens=1024,
    openai_api_version=aoai_api_version,
    azure_deployment=aoai_deployment_name                    
)

parser = JsonOutputParser(pydantic_object=QAPair)
prompt = get_qna_prompt_template()
#prompt = get_qna_repair_cost_prompt_template()
chain = prompt | llm | parser

In [ ]:
input_batch = []

for doc in mixed_chunks:
    dic = {"context": doc, "domain": "Samsung Galaxy S series Smartphone, especially S23 and S24 series", "num_questions": "3"}
    input_batch.append(dic)

for doc in text_chunks:
    dic = {"context": doc, "domain": "Samsung Galaxy S series Smartphone, especially S23 and S24 series", "num_questions": "3"}
    input_batch.append(dic)

for doc in image_summaries:
    dic = {"context": doc, "domain": "Samsung Galaxy S series Smartphone, especially S23 and S24 series", "num_questions": "3"}
    input_batch.append(dic)        


In [ ]:
%%time
#input_query = {"context": md_content, "domain": "Samsung Galaxy S series Smartphone", "num_questions": "3"}
qa_pair = chain.batch(input_batch, {"max_concurrency": 5})

## 3. Save to jsonl for fine-tuning
---

In [ ]:
import json
from util.common_utils import convert_to_oai_format, save_jsonl

output_dir = './dataset'
os.makedirs(output_dir, exist_ok=True)

system_prompt_msg = """You are an AI assistant that provides guidance to help users self-service resolve abnormalities in their Galaxy mobile phone.\n
Please answer the questions accurately. If the question is in Korean, write your answer in Korean. If the question is in English, write your answer in English."""

save_filename = "cs-self-solve"
oai_qa_pair = convert_to_oai_format(question_answer_list)

#save_jsonl(qa_pair, f"{output_dir}/{save_filename}.jsonl")
save_jsonl(oai_qa_pair, f"{output_dir}/{save_filename}-oai.jsonl")

### Clean up

In [ ]:
!rm -rf {splitted_raw_data_dir} pdf_image_tmp pdf_mixed_tmp outputs_tmp images